In [2]:
# first we import all relevant packages
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import pynbody
from scipy import stats
from scipy.stats import binned_statistic
import pickle
import pandas as pd

pynbody.config['halo-class-priority'] =  [pynbody.halo.ahf.AHFCatalogue,
                                          pynbody.halo.GrpCatalogue,
                                          pynbody.halo.AmigaGrpCatalogue,
                                          pynbody.halo.legacy.RockstarIntermediateCatalogue,
                                          pynbody.halo.rockstar.RockstarCatalogue,
                                          pynbody.halo.subfind.SubfindCatalogue, pynbody.halo.hop.HOPCatalogue]

In [3]:
# we set the global matplotlib parameters so the fonts are all consistent and serif 
mpl.rc('font',**{'family':'serif','monospace':['Palatino']})
mpl.rc('text', usetex=True)
mpl.rcParams.update({'font.size': 9})
mpl.rcParams['hatch.linewidth'] = 0.8

In [4]:
def read_timescales():
    '''Function to read in the resulting data file which contains quenching and infall times'''
    data = []
    with open('../../Data/QuenchingTimescales.data', 'rb') as f:
        while True:
            try:
                data.append(pickle.load(f,encoding='latin1'))
            except EOFError:
                break

    data = pd.DataFrame(data)
    return data

In [5]:
def read_z0(simname):
    '''Function to read in the data file which contains z=0 information for the various halos'''
    data = []
    with open('../../Data/z0_data/'+ simname + '.data', 'rb') as f:
        while True:
            try:
                data.append(pickle.load(f,encoding='latin1'))
            except EOFError:
                break

    data1 = pd.DataFrame(data)
    return data1

In [24]:
timescales = read_timescales()
quenched = np.array(timescales.quenched, dtype=bool)
tau = np.array(timescales.tinfall) - np.array(timescales.tquench)
quench_after_infall = tau > 0

resolved = np.array(timescales.n_star, dtype=int) > 100
satellites = ~np.isnan(np.array(timescales.tinfall))

timescales = timescales[quenched & quench_after_infall]

len(timescales)

/Users/hollis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in greater
  after removing the cwd from sys.path.


22

In [41]:
timescales = read_timescales()
quenched = np.array(timescales.quenched, dtype=bool)
tau = np.array(timescales.tinfall) - np.array(timescales.tquench)
quench_after_infall = tau > 0

resolved = np.array(timescales.n_star, dtype=int) > 100
satellites = ~np.isnan(np.array(timescales.tinfall))

timescales = timescales[quenched & satellites & resolved]

print(f'There are {len(timescales)} resolved satellites that are quenched at z=0.')

gas_masses = np.array([])
sims, haloids = np.array(timescales.sim), np.array(timescales.haloid)
for sim, haloid in zip(sims,haloids):
    d = read_z0(sim)
    d = d[d.haloid==haloid]
    Mgas = d.M_gas.tolist()[0]
    gas_masses = np.append(gas_masses, Mgas)

print(f'Of these {len(timescales)}, only {len(gas_masses[gas_masses != 0])} have gas at z=0.')
print(fr"These are:")
print(*[str(s)+'-'+str(h) for s,h in zip(sims[gas_masses != 0], haloids[gas_masses != 0])])
print('(With the caveat that our haloids might be different from what you are using, since we re-ran AHF a while back to use the background density to get Rvir)')


/Users/hollis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in greater
  after removing the cwd from sys.path.


There are 35 resolved satellites that are quenched at z=0.
Of these 35, only 8 have gas at z=0.
These are:
h148-13 h148-28 h148-37 h148-68 h242-40 h242-80 h229-20 h229-23
(With the caveat that our haloids might be different from what you are using, since we re-ran AHF a while back to use the background density to get Rvir)


In [11]:
data = read_z0('h148')

In [12]:
data = data[data.haloid == 1]
Mstar = data.M_star.tolist()[0]
print(f'{Mstar:.2e}')

1.93e+11


In [16]:
mass = data.mass.tolist()[0]
print(f'{mass:.2e}')

2.66e+12


In [17]:
data = read_z0('h329')
print(np.unique(data.haloid))

[   1   11   31   33   40   64  103  133  137  146  185  447  729  996
 1509]
